<a href="https://colab.research.google.com/github/pascuapablo/CEIA/blob/master/scripts/computer_vision/tp/TP_CVII_descarga_de_imagenes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentinelhub
!pip install libtiff
!pip install tifffile

import tifffile as t
from libtiff import TIFF
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as math
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt

import matplotlib.colors as colors
import pickle as pkl
import os
import cv2 as cv
from glob import glob

from sentinelhub import MimeType, CRS, BBox, SentinelHubRequest, SentinelHubDownloadClient, DataCollection, bbox_to_dimensions, DownloadRequest




In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# degrees to radians
def deg2rad(degrees):
    return math.pi*degrees/180.0
# radians to degrees
def rad2deg(radians):
    return 180.0*radians/math.pi

# Semi-axes of WGS-84 geoidal reference
WGS84_a = 6378137.0  # Major semiaxis [m]
WGS84_b = 6356752.3  # Minor semiaxis [m]

# Earth radius at a given latitude, according to the WGS-84 ellipsoid [m]
def WGS84EarthRadius(lat):
    # http://en.wikipedia.org/wiki/Earth_radius
    An = WGS84_a*WGS84_a * math.cos(lat)
    Bn = WGS84_b*WGS84_b * math.sin(lat)
    Ad = WGS84_a * math.cos(lat)
    Bd = WGS84_b * math.sin(lat)
    return math.sqrt( (An*An + Bn*Bn)/(Ad*Ad + Bd*Bd) )

# Bounding box surrounding the point at given coordinates,
# assuming local approximation of Earth surface as a sphere
# of radius given by WGS84
def boundingBox(latitudeInDegrees, longitudeInDegrees, halfSideInMeters):
    lat = deg2rad(latitudeInDegrees)
    lon = deg2rad(longitudeInDegrees)
    halfSide = halfSideInMeters

    # Radius of Earth at given latitude
    radius = WGS84EarthRadius(lat)
    # Radius of the parallel at given latitude
    pradius = radius*math.cos(lat)

    latMin = lat - halfSide/radius
    latMax = lat + halfSide/radius
    lonMin = lon - halfSide/pradius
    lonMax = lon + halfSide/pradius

    return (rad2deg(latMin), rad2deg(lonMin), rad2deg(latMax), rad2deg(lonMax))



In [ ]:

from sentinelhub import SHConfig

# In case you put the credentials into the configuration file you can leave this unchanged
# GMAIL
CLIENT_ID = '370f2a91-39ba-492b-9458-14631de73771'
CLIENT_SECRET = 'Y7CjU?s:|9!SM~mS!GGQNqoILnx.I[IIRuUZ6+-W'
# HOTMAIL
CLIENT_ID = '91902aba-2792-4804-a1bd-513397611453'
CLIENT_SECRET = 'QI;&G8I-N;:U%&r.w1menl[kCuOC)#UCV~;jzP;f'
config = SHConfig()

if CLIENT_ID and CLIENT_SECRET:
    config.sh_client_id = CLIENT_ID
    config.sh_client_secret = CLIENT_SECRET

if config.sh_client_id == '' or config.sh_client_secret == '':
    print("Warning! To use Sentinel Hub services, please provide the credentials (client ID and client secret).")


In [ ]:

def downloadImage(cultivo,lat,long ,from_date, to_date, maxCloudPercentage=0.5):
  
  resolution = 10 # en metros
  point_lat_lon = (lat,long)
  radius_in_meters = 80


  geo_position_box = boundingBox(point_lat_lon[0],point_lat_lon[1],radius_in_meters)



  box_in_wgs84 = [geo_position_box[1],
                  geo_position_box[0],
                  geo_position_box[3],
                  geo_position_box[2]]


  bbox = BBox(bbox=box_in_wgs84, crs=CRS.WGS84)
  bbox_size_in_pixels = bbox_to_dimensions(bbox, resolution=resolution)

  evalscript_all_bands = """
      //VERSION=3
      function setup() {
          return {
              input: [{
                  bands: ["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B10","B11","B12"],
                  units: "DN"
              }],
              output: {
                  bands: 13,
                  sampleType: "INT16"
              },
          };
      }

      function evaluatePixel(sample) {
      
          return [sample.B01,
                  sample.B02,
                  sample.B03,
                  sample.B04,
                  sample.B05,
                  sample.B06,
                  sample.B07,
                  sample.B08,
                  sample.B8A,
                  sample.B09,
                  sample.B10,
                  sample.B11,
                  sample.B12];
      }
  """

  request_true_color = SentinelHubRequest(
      evalscript=evalscript_all_bands,
      input_data=[
          SentinelHubRequest.input_data(
              data_collection=DataCollection.SENTINEL2_L1C,
              time_interval=(from_date, to_date),
              mosaicking_order='leastCC',
              maxcc=maxCloudPercentage
              
          )
      ],
      responses=[
          SentinelHubRequest.output_response('default', MimeType.TIFF)
      ],
      bbox=bbox,
      size=bbox_size_in_pixels,
      config=config
  )
  all_bands_image = request_true_color.get_data()
  all_bands_image = all_bands_image[0]


  return all_bands_image


def saveImage(path, fileName,image):
  if  not os.path.isdir(path):
    os.makedirs(path)

  filePath = os.path.join(path, fileName)
  t.imwrite(filePath, image, photometric='minisblack')

  return

In [ ]:
# Cargar dataset de train
train_data=pd.read_csv('https://raw.githubusercontent.com/hernancontigiani/desafiosAgTech2020/main/df_data_train.csv')

train_github_files= [
                     'https://raw.githubusercontent.com/hernancontigiani/desafiosAgTech2020/main/preprocesamiento/df_train_A.csv',
                     'https://raw.githubusercontent.com/hernancontigiani/desafiosAgTech2020/main/preprocesamiento/df_train_G.csv',
                     'https://raw.githubusercontent.com/hernancontigiani/desafiosAgTech2020/main/preprocesamiento/df_train_M.csv',
                     'https://raw.githubusercontent.com/hernancontigiani/desafiosAgTech2020/main/preprocesamiento/df_train_R.csv',
                     'https://raw.githubusercontent.com/hernancontigiani/desafiosAgTech2020/main/preprocesamiento/df_train_S.csv',
                     'https://raw.githubusercontent.com/hernancontigiani/desafiosAgTech2020/main/preprocesamiento/df_train_T.csv',
                     'https://raw.githubusercontent.com/hernancontigiani/desafiosAgTech2020/main/preprocesamiento/df_train_U.csv',
                     'https://raw.githubusercontent.com/hernancontigiani/desafiosAgTech2020/main/preprocesamiento/df_train_X.csv',
                     'https://raw.githubusercontent.com/hernancontigiani/desafiosAgTech2020/main/preprocesamiento/df_train_aa.csv',
                     'https://raw.githubusercontent.com/hernancontigiani/desafiosAgTech2020/main/preprocesamiento/df_train_m.csv',
                     'https://raw.githubusercontent.com/hernancontigiani/desafiosAgTech2020/main/preprocesamiento/df_train_s.csv',
                     'https://raw.githubusercontent.com/hernancontigiani/desafiosAgTech2020/main/preprocesamiento/df_train_B.csv',
                     'https://raw.githubusercontent.com/hernancontigiani/desafiosAgTech2020/main/preprocesamiento/df_train_N.csv',
                     'https://raw.githubusercontent.com/hernancontigiani/desafiosAgTech2020/main/preprocesamiento/df_train_P.csv',                     
                     ]



download_points = None
for file in train_github_files:
  if download_points is None:
    download_points = pd.read_csv(file)
  else:
    aux = pd.read_csv(file)
    download_points = pd.concat([download_points, aux])

merged_inner = pd.merge(how='left',left=download_points, right=train_data, left_on='GlobalId', right_on='GlobalId')

merged_inner = merged_inner.rename(columns={
    'Latitud_y': 'Latitud',
    'Longitud_y':'Longitud',
    'Cultivo_y':'Cultivo',
    'Campania_y': 'Campania'})

merged_inner['Cultivo'] = merged_inner['Cultivo'].apply(lambda x : 'M' if 'S/M' in x else x)
merged_inner.head()


augmented_data = pd.read_csv('https://raw.githubusercontent.com/hernancontigiani/desafiosAgTech2020/main/preprocesamiento/df_train_augmented.csv')
merged_inner = pd.concat([augmented_data, merged_inner])

merged_inner.head()




,GlobalId,Cultivo,Latitud,Campania,best_q,best_q2,Longitud
0,1451A,A,-34.342117,19/20,15,16,-62.256167
1,1451B,A,-34.306912,19/20,15,16,-62.289305
2,1451C,A,-34.310234,19/20,15,16,-62.196488
3,1452A,A,-33.941216,19/20,15,14,-61.504764
4,1452B,A,-33.963033,19/20,15,14,-61.494374


In [ ]:
def addYearToDateAccordingToCapaing(campaing,date):
  if '18/19' in campaing:
    return '2019-' + date
  else:
    return '2020-' + date

def generate_date(best_q,campaing):
  dates = {
    1: ["01-01", "01-02", "01-03", "01-04", "01-05", "01-06", "01-07", "01-08", "01-09", "01-10", "01-11", "01-12", "01-13", "01-14", "01-15"],
    2: ["01-16", "01-17", "01-18", "01-19", "01-20", "01-21", "01-22", "01-23", "01-24", "01-25", "01-26", "01-27", "01-28", "01-29", "01-30"],
    3: ["02-01", "02-02", "02-03", "02-04", "02-05", "02-06", "02-07", "02-08", "02-09", "02-10", "02-11", "02-12", "02-13", "02-14", "02-15"],
    4: ["02-16", "02-17", "02-18", "02-19", "02-20", "02-21", "02-22", "02-23", "02-24", "02-25", "02-26", "02-27", "02-28"],
    5: ["03-01", "03-02", "03-03", "03-04", "03-05", "03-06", "03-07", "03-08", "03-09", "03-10", "03-11", "03-12", "03-13", "03-14", "03-15"],
    6: ["03-16", "03-17", "03-18", "03-19", "03-20", "03-21", "03-22", "03-23", "03-24", "03-25", "03-26", "03-27", "03-28", "03-29", "03-30"],
    7: ["04-01", "04-02", "04-03", "04-04", "04-05", "04-06", "04-07", "04-08", "04-09", "04-10", "04-11", "04-12", "04-13", "04-14", "04-15"],
    8: ["04-16", "04-17", "04-18", "04-19", "04-20", "04-21", "04-22", "04-23", "04-24", "04-25", "04-26", "04-27", "04-28", "04-29", "04-30"],
    9: ["05-01", "05-02", "05-03", "05-04", "05-05", "05-06", "05-07", "05-08", "05-09", "05-10", "05-11", "05-12", "05-13", "05-14", "05-15"],
    10: ["05-16", "05-17", "05-18", "05-19", "05-20", "05-21", "05-22", "05-23", "05-24", "05-25", "05-26", "05-27", "05-28", "05-29", "05-30"],
    11: ["06-01", "06-02", "06-03", "06-04", "06-05", "06-06", "06-07", "06-08", "06-09", "06-10", "06-11", "06-12", "06-13", "06-14", "06-15"],
    12: ["06-16", "06-17", "06-18", "06-19", "06-20", "06-21", "06-22", "06-23", "06-24", "06-25", "06-26", "06-27", "06-28", "06-29", "06-30"],
    13: ["07-01", "07-02", "07-03", "07-04", "07-05", "07-06", "07-07", "07-08", "07-09", "07-10", "07-11", "07-12", "07-13", "07-14", "07-15"],
    14: ["07-16", "07-17", "07-18", "07-19", "07-20", "07-21", "07-22", "07-23", "07-24", "07-25", "07-26", "07-27", "07-28", "07-29", "07-30"],
    15: ["08-01", "08-02", "08-03", "08-04", "08-05", "08-06", "08-07", "08-08", "08-09", "08-10", "08-11", "08-12", "08-13", "08-14", "08-15"],
    16: ["08-16", "08-17", "08-18", "08-19", "08-20", "08-21", "08-22", "08-23", "08-24", "08-25", "08-26", "08-27", "08-28", "08-29", "08-30"],
    17: ["09-01", "09-02", "09-03", "09-04", "09-05", "09-06", "09-07", "09-08", "09-09", "09-10", "09-11", "09-12", "09-13", "09-14", "09-15"],
    18: ["09-16", "09-17", "09-18", "09-19", "09-20", "09-21", "09-22", "09-23", "09-24", "09-25", "09-26", "09-27", "09-28", "09-29", "09-30"],
    19: ["10-01", "10-02", "10-03", "10-04", "10-05", "10-06", "10-07", "10-08", "10-09", "10-10", "10-11", "10-12", "10-13", "10-14", "10-15"],
    20: ["10-16", "10-17", "10-18", "10-19", "10-20", "10-21", "10-22", "10-23", "10-24", "10-25", "10-26", "10-27", "10-28", "10-29", "10-30"],
    21: ["11-01", "11-02", "11-03", "11-04", "11-05", "11-06", "11-07", "11-08", "11-09", "11-10", "11-11", "11-12", "11-13", "11-14", "11-15"],
    22: ["11-16", "11-17", "11-18", "11-19", "11-20", "11-21", "11-22", "11-23", "11-24", "11-25", "11-26", "11-27", "11-28", "11-29", "11-30"],
    23: ["12-01", "12-02", "12-03", "12-04", "12-05", "12-06", "12-07", "12-08", "12-09", "12-10", "12-11", "12-12", "12-13", "12-14", "12-15"],
    24: ["12-16", "12-17", "12-18", "12-19", "12-20", "12-21", "12-22", "12-23", "12-24", "12-25", "12-26", "12-27", "12-28", "12-29", "12-30"],
  }

  datesToReturn = {}
  for i in range(len(dates[best_q])-1):
    from_date = addYearToDateAccordingToCapaing(campaing,dates[best_q][i])
    to_date = addYearToDateAccordingToCapaing(campaing,dates[best_q][i+1])

    datesToReturn[i]= (from_date,to_date)
  
  return datesToReturn






In [ ]:
savePath = '/content/drive/MyDrive/imagenes_tp_v4'


for index, row in merged_inner.iterrows():
  datesDict = generate_date(row['best_q2'],row['Campania'])
  cultivo = row['Cultivo']
  maxValuePixelInImage = []

  for day in datesDict.keys():

    print(str(index) + '/' + str(len(merged_inner)), end = ' ')
    image = downloadImage(
                  cultivo = cultivo,
                  lat = row['Latitud'],
                  long = row['Longitud'],
                  from_date= datesDict[day][0],            
                  to_date= datesDict[day][1],
                  maxCloudPercentage=0.3
                  )
    
  

    true_color_image = image[:,:,[3,2,1]]
    
    saveFullPath = os.path.join(savePath,cultivo)
    imageName = cultivo + '_' + str(row["GlobalId"]) + '-D'+str(day)+'-q2.tif'

    if true_color_image.max() > 0.0001 :
      maxValue = true_color_image.max()

      if maxValue not in maxValuePixelInImage:
        print(imageName, true_color_image.max(),true_color_image.min(),true_color_image.mean(), image.mean())
        saveImage(saveFullPath,imageName, image)    
        #plt.imshow(true_color_image*0.00035)
      else:
        print(imageName,"no guardo por que es igual a la anterior")
      
      maxValuePixelInImage.append(maxValue)
    else: 
      print(imageName,"no guardo porque no hay data")

    

  


0/38 A_1451A-D0-q2.tif 876 393 661.3919270833334 397.47415865384613
0/38 A_1451A-D1-q2.tif no guardo por que es igual a la anterior
0/38 A_1451A-D2-q2.tif 1036 511 793.5625 500.31580528846155
0/38 A_1451A-D3-q2.tif no guardo por que es igual a la anterior
0/38 A_1451A-D4-q2.tif no guardo porque no hay data
0/38 A_1451A-D5-q2.tif no guardo porque no hay data
0/38 A_1451A-D6-q2.tif no guardo porque no hay data
0/38 A_1451A-D7-q2.tif no guardo porque no hay data
0/38 A_1451A-D8-q2.tif no guardo porque no hay data
0/38 A_1451A-D9-q2.tif no guardo porque no hay data
0/38 A_1451A-D10-q2.tif 890 408 678.2955729166666 404.84945913461536
0/38 A_1451A-D11-q2.tif no guardo por que es igual a la anterior
0/38 A_1451A-D12-q2.tif no guardo porque no hay data
0/38 A_1451A-D13-q2.tif no guardo porque no hay data
1/38 A_1451B-D0-q2.tif 871 378 649.3580729166666 391.86868990384613
1/38 A_1451B-D1-q2.tif no guardo por que es igual a la anterior
1/38 A_1451B-D2-q2.tif no guardo porque no hay data
1/38 A_1